# Credit card fraud detector

## Introduction

It is important that credit card companies are able to recognize fraudulent credit card transactions so that customers are not charged for items that they did not purchase.

The datasets contains transactions made by credit cards in September 2013 by european cardholders. This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions. The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.

It contains only numerical input variables which are the result of a PCA transformation. Unfortunately, due to confidentiality issues, we cannot provide the original features and more background information about the data. Features V1, V2, ... V28 are the principal components obtained with PCA, the only features which have not been transformed with PCA are 'Time' and 'Amount'. Feature 'Time' contains the seconds elapsed between each transaction and the first transaction in the dataset. The feature 'Amount' is the transaction Amount, this feature can be used for example-dependant cost-senstive learning. Feature 'Class' is the response variable and it takes value 1 in case of fraud and 0 otherwise. 

More details about the dataset can be found here - https://www.kaggle.com/mlg-ulb/creditcardfraud

## Investigate and process the data

Let's start by downloading and reading in the credit card fraud data set.

In [1]:
%%bash
wget https://s3-us-west-2.amazonaws.com/sagemaker-e2e-solutions/fraud-detection/creditcardfraud.zip
unzip creditcardfraud.zip

Archive:  creditcardfraud.zip
  inflating: creditcard.csv          


--2020-02-09 20:01:46--  https://s3-us-west-2.amazonaws.com/sagemaker-e2e-solutions/fraud-detection/creditcardfraud.zip
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.218.225.0
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.218.225.0|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 69155632 (66M) [application/zip]
Saving to: ‘creditcardfraud.zip’

     0K .......... .......... .......... .......... ..........  0%  331K 3m24s
    50K .......... .......... .......... .......... ..........  0% 70.5M 1m42s
   100K .......... .......... .......... .......... ..........  0%  663K 1m42s
   150K .......... .......... .......... .......... ..........  0% 86.9M 77s
   200K .......... .......... .......... .......... ..........  0%  669K 81s
   250K .......... .......... .......... .......... ..........  0%  142M 68s
   300K .......... .......... .......... .......... ..........  0% 59.5M 58s
   350K .......... ..........

In [2]:
import numpy as np 
import pandas as pd

data = pd.read_csv('creditcard.csv', delimiter=',')

Let's take a peek at our data (we only show a subset of the columns in the table):

In [3]:
print(data.columns)
data[['Time', 'V1', 'V2', 'V27', 'V28', 'Amount', 'Class']].describe()

Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class'],
      dtype='object')


,Time,V1,V2,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,3.919560e-15,5.688174e-16,-3.660161e-16,-1.206049e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,3.161220e+01,3.384781e+01,25691.160000,1.000000


The class column corresponds to whether or not a transaction is fradulent. We see that the majority of data is non-fraudulant with only $492$ ($.173\%$) of the data corresponding to fraudulant examples.

In [4]:
nonfrauds, frauds = data.groupby('Class').size()
print('Number of frauds: ', frauds)
print('Number of non-frauds: ', nonfrauds)
print('Percentage of fradulent data:', 100.*frauds/(frauds + nonfrauds))

Number of frauds:  492
Number of non-frauds:  284315
Percentage of fradulent data: 0.1727485630620034


This dataset has 28 columns, $V_i$ for $i=1..28$ of anonymized features along with columns for time, amount, and class. We already know that the columns $V_i$ have been normalized to have $0$ mean and unit standard deviation as the result of a PCA. You can read more about PCA here:. 

Tip: For our dataset this amount of preprocessing will give us reasonable accuracy, but it's important to note that there are more preprocessing steps one can use to improve accuracy . For unbalanced data sets like ours where the positive (fraudulent) examples occur much less frequently than the negative (legitimate) examples, we may try “over-sampling” the minority dataset by generating synthetic data (read about SMOTE in Data Mining for Imbalanced Datasets: An Overview (https://link.springer.com/chapter/10.1007%2F0-387-25465-X_40) or undersampling the majority class by using ensemble methods (see http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.68.6858&rep=rep1&type=pdfor).

### Split input dataset into train, validate, batch

In [6]:
#data split in three sets, training, validation and batch inference
rand_split = np.random.rand(len(data))
train_list = rand_split < 0.8
val_list = (rand_split >= 0.8) & (rand_split < 0.9)
batch_list = rand_split >= 0.9

data_train = data[train_list]
data_val = data[val_list]
data_batch = data[batch_list].drop(['Class'],axis=1)


In [7]:
feature_columns = data_train.columns[:-1]
print(feature_columns)
label_column = data_train.columns[-1]
print(label_column)

features = data_train[feature_columns].values.astype('float32')
labels = (data_train[label_column].values).astype('float32')

Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount'],
      dtype='object')
Class


Let's do some analysis and discuss different ways we can preprocess our data. Let's discuss the way in which this data was preprocessed.

## SageMaker Linear Learner

### Prepare Data and Upload to S3

The Amazon common libraries provide utilities to convert NumPy n-dimensional arrays into a the Record-IO format which SageMaker uses for a concise representation of features and labels. The Record-IO format is implemented via protocol buffer so the serialization is very efficient.

In [8]:
import io
import sagemaker.amazon.common as smac

buf = io.BytesIO()
smac.write_numpy_to_dense_tensor(buf, features, labels)
buf.seek(0);

Now we upload the data to S3 using boto3.

In [10]:
import boto3
import os
import sagemaker

role = sagemaker.get_execution_role()
sess = sagemaker.Session()
bucket = sess.default_bucket()

prefix = 'fraud-detection-lab-v2'
key = 'recordio-pb-data'
boto3.resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train', key)).upload_fileobj(buf)

s3_train_data = 's3://{}/{}/train/{}'.format(bucket, prefix, key)
print('Uploaded training data location: {}'.format(s3_train_data))

output_location = 's3://{}/{}/output'.format(bucket, prefix)
print('Training artifacts will be uploaded to: {}'.format(output_location))

Uploaded training data location: s3://sagemaker-us-east-1-716664005094/fraud-detection-lab-v2/train/recordio-pb-data
Training artifacts will be uploaded to: s3://sagemaker-us-east-1-716664005094/fraud-detection-lab-v2/output


Now we train a Linear Learner using SageMaker's built-in algorithm. To specify the Linear Learner algorithm, we use a utility function to obtain it's URI. A complete list of build-in algorithms is found here: https://docs.aws.amazon.com/sagemaker/latest/dg/algos.html

In [11]:
from sagemaker.amazon.amazon_estimator import get_image_uri

container = get_image_uri(boto3.Session().region_name, 'linear-learner')

SageMaker abstracts training with Estimators. We can pass container, and all parameters to the estimator, as well as the hyperparameters for the linear learner and fit the estimator to the data in S3.

In [12]:
from sagemaker import get_execution_role

print(features.shape[1])

linear = sagemaker.estimator.Estimator(container,
                                       get_execution_role(), 
                                       train_instance_count=1, 
                                       train_instance_type='ml.c4.xlarge',
                                       output_path=output_location,
                                       sagemaker_session=session)
linear.set_hyperparameters(feature_dim=features.shape[1],
                           predictor_type='binary_classifier',
                           mini_batch_size=200)

linear.fit({'train': s3_train_data})

30
2020-02-09 20:08:59 Starting - Starting the training job...
2020-02-09 20:09:01 Starting - Launching requested ML instances......
2020-02-09 20:10:06 Starting - Preparing the instances for training......
2020-02-09 20:11:28 Downloading - Downloading input data
2020-02-09 20:11:28 Training - Downloading the training image...
2020-02-09 20:11:47 Training - Training image download completed. Training in progress.Docker entrypoint called with argument(s): train
[02/09/2020 20:11:51 INFO 139820100958016] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-input.json: {u'loss_insensitivity': u'0.01', u'epochs': u'15', u'feature_dim': u'auto', u'init_bias': u'0.0', u'lr_scheduler_factor': u'auto', u'num_calibration_samples': u'10000000', u'accuracy_top_k': u'3', u'_num_kv_servers': u'auto', u'use_bias': u'true', u'num_point_for_scaler': u'10000', u'_log_level': u'info', u'quantile': u'0.5', u'bias_lr_mult': u'auto', u'lr_scheduler_step': u

[2020-02-09 20:12:53.825] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 5, "duration": 31339, "num_examples": 1142, "num_bytes": 38357424}
#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.007676145257436451, "sum": 0.007676145257436451, "min": 0.007676145257436451}}, "EndTime": 1581279173.825863, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 1}, "StartTime": 1581279173.825764}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.00617980948824837, "sum": 0.00617980948824837, "min": 0.00617980948824837}}, "EndTime": 1581279173.825964, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 1}, "StartTime": 1581279173.825943}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.00787662049789

[2020-02-09 20:13:24.708] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 7, "duration": 30875, "num_examples": 1142, "num_bytes": 38357424}
#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.006565652294092592, "sum": 0.006565652294092592, "min": 0.006565652294092592}}, "EndTime": 1581279204.708366, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 2}, "StartTime": 1581279204.70828}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.005156582432866788, "sum": 0.005156582432866788, "min": 0.005156582432866788}}, "EndTime": 1581279204.708462, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 2}, "StartTime": 1581279204.708442}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.006673377065

[2020-02-09 20:13:55.770] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 9, "duration": 31056, "num_examples": 1142, "num_bytes": 38357424}
#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.006079726430255263, "sum": 0.006079726430255263, "min": 0.006079726430255263}}, "EndTime": 1581279235.771023, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 3}, "StartTime": 1581279235.770935}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.004875390046618154, "sum": 0.004875390046618154, "min": 0.004875390046618154}}, "EndTime": 1581279235.771111, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 3}, "StartTime": 1581279235.771091}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.00613683051

[2020-02-09 20:14:26.615] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 11, "duration": 30838, "num_examples": 1142, "num_bytes": 38357424}
#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.005885539412887001, "sum": 0.005885539412887001, "min": 0.005885539412887001}}, "EndTime": 1581279266.61583, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 4}, "StartTime": 1581279266.615739}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.0046822449945543515, "sum": 0.0046822449945543515, "min": 0.0046822449945543515}}, "EndTime": 1581279266.615912, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 4}, "StartTime": 1581279266.615893}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.00589911

...

[2020-02-09 20:18:02.544] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 25, "duration": 30772, "num_examples": 1142, "num_bytes": 38357424}
#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.005657115770760176, "sum": 0.005657115770760176, "min": 0.005657115770760176}}, "EndTime": 1581279482.544455, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 11}, "StartTime": 1581279482.54436}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.004332891492651389, "sum": 0.004332891492651389, "min": 0.004332891492651389}}, "EndTime": 1581279482.544538, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 11}, "StartTime": 1581279482.544518}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.004879277

[2020-02-09 20:18:33.588] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 27, "duration": 31037, "num_examples": 1142, "num_bytes": 38357424}
#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.005634759075418898, "sum": 0.005634759075418898, "min": 0.005634759075418898}}, "EndTime": 1581279513.588162, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 12}, "StartTime": 1581279513.588071}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.004298826978255413, "sum": 0.004298826978255413, "min": 0.004298826978255413}}, "EndTime": 1581279513.588262, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 12}, "StartTime": 1581279513.58824}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.004799011

[2020-02-09 20:19:04.539] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 29, "duration": 30945, "num_examples": 1142, "num_bytes": 38357424}
#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.005612872846197666, "sum": 0.005612872846197666, "min": 0.005612872846197666}}, "EndTime": 1581279544.540066, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 13}, "StartTime": 1581279544.539987}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.004257875365636897, "sum": 0.004257875365636897, "min": 0.004257875365636897}}, "EndTime": 1581279544.540145, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 13}, "StartTime": 1581279544.540126}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.00473038

[2020-02-09 20:19:35.481] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 31, "duration": 30934, "num_examples": 1142, "num_bytes": 38357424}
#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.005591645949347945, "sum": 0.005591645949347945, "min": 0.005591645949347945}}, "EndTime": 1581279575.481151, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 14}, "StartTime": 1581279575.481073}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.0042371542432152005, "sum": 0.0042371542432152005, "min": 0.0042371542432152005}}, "EndTime": 1581279575.481226, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 14}, "StartTime": 1581279575.481213}

#metrics {"Metrics": {"train_binary_classification_cross_entropy_objective": {"count": 1, "max": 0.00467


2020-02-09 20:19:49 Uploading - Uploading generated training model
2020-02-09 20:19:49 Completed - Training job completed
Training seconds: 515
Billable seconds: 515


### Host Linear Learner

Now we deploy the estimator to and endpoint.

In [14]:
from sagemaker.predictor import csv_serializer, json_deserializer

linear_predictor = linear.deploy(initial_instance_count=1,
                                 endpoint_name="fraud-detection-endpoint-v2",
                                 instance_type='ml.m4.xlarge')
# Specify input and output formats.
linear_predictor.content_type = 'text/csv'
linear_predictor.serializer = csv_serializer
linear_predictor.deserializer = json_deserializer

Using already existing model: linear-learner-2020-02-09-20-08-59-743


-------------------!

### Predict - Realtime 

In [15]:
import json
import os
import boto3
import random
import datetime
import re



def get_data():
    
    non_fraud_example = [1.00000000e+00, -9.66271698e-01, -1.85226008e-01, 1.79299331e+00, -8.63291264e-01, -1.03088794e-02, 1.24720311e+00, 2.37608939e-01,
                         3.77435863e-01, -1.38702404e+00, -5.49519211e-02, -2.26487264e-01, 1.78228229e-01, 5.07756889e-01, -2.87923753e-01, -6.31418109e-01,
                         -1.05964720e+00, -6.84092760e-01, 1.96577501e+00, -1.23262203e+00, -2.08037779e-01, -1.08300455e-01, 5.27359685e-03, -1.90320522e-01,
                         -1.17557538e+00, 6.47376060e-01, -2.21928850e-01, 6.27228469e-02, 6.14576302e-02, 1.23500000e+02]
    
    fraud_example = [4.0600000e+02, -2.3122265e+00, 1.9519920e+00, -1.6098508e+00, 3.9979055e+00, -5.2218789e-01, -1.4265453e+00, -2.5373874e+00,
                     1.3916572e+00, -2.7700894e+00, -2.7722721e+00, 3.2020333e+00, -2.8999074e+00, -5.9522188e-01, -4.2892537e+00, 3.8972411e-01, -1.1407472e+00,
                     -2.8300557e+00, -1.6822468e-02, 4.1695571e-01, 1.2691055e-01, 5.1723236e-01, -3.5049368e-02, -4.6521106e-01, 3.2019821e-01, 4.4519167e-02,
                     1.7783980e-01, 2.6114500e-01, -1.4327587e-01, 0.0000000e+00]
    
    examples = [fraud_example, non_fraud_example]
    
    idx = 0
    
    rand = random.random()

    print('\nrand:', rand)
    
    if rand > 0.5:
        idx = 1
    
    return ','.join(map(str, examples[idx]))

def get_fraud_prediction(data):
    sagemaker_endpoint_name = 'fraud-detection-endpoint-v2'
    sagemaker_runtime = boto3.client('sagemaker-runtime')
    response = sagemaker_runtime.invoke_endpoint(EndpointName=sagemaker_endpoint_name, ContentType='text/csv',
                                                 Body=data)
    print('\nresponse:', response)
    result = json.loads(response['Body'].read().decode())
    print('\nresult:', result)
    pred = int(result['predictions'][0]['predicted_label'])
    return pred


data_payload = get_data()
print(data_payload)
pred = get_fraud_prediction(data_payload)
print('\nprediction:', pred)





rand: 0.6084098239404958
1.0,-0.966271698,-0.185226008,1.79299331,-0.863291264,-0.0103088794,1.24720311,0.237608939,0.377435863,-1.38702404,-0.0549519211,-0.226487264,0.178228229,0.507756889,-0.287923753,-0.631418109,-1.0596472,-0.68409276,1.96577501,-1.23262203,-0.208037779,-0.108300455,0.00527359685,-0.190320522,-1.17557538,0.64737606,-0.22192885,0.0627228469,0.0614576302,123.5

response: {'ResponseMetadata': {'RequestId': '651551a4-7ae1-4391-b36c-e245b0e9d631', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '651551a4-7ae1-4391-b36c-e245b0e9d631', 'x-amzn-invoked-production-variant': 'AllTraffic', 'date': 'Sun, 9 Feb 2020 20:52:13 GMT', 'content-type': 'application/json', 'content-length': '75'}, 'RetryAttempts': 0}, 'ContentType': 'application/json', 'InvokedProductionVariant': 'AllTraffic', 'Body': <botocore.response.StreamingBody object at 0x7f79fc5c1400>}

result: {'predictions': [{'score': 6.844831659691408e-05, 'predicted_label': 0.0}]}

prediction: 0


### Batch Predictions

In [26]:
print(bucket)
print(prefix)
batch_file = 'batch_data.csv'
data_batch.to_csv(batch_file,index=False,header=False)
sess.upload_data(batch_file, key_prefix='{}/batch'.format(prefix))

sagemaker-us-east-1-716664005094
fraud-detection-lab-v2


's3://sagemaker-us-east-1-716664005094/fraud-detection-lab-v2/batch/batch_data.csv'

In [27]:
%%time





sm_transformer = linear.transformer(1, 'ml.m4.xlarge')

# start a transform job
input_location = 's3://{}/{}/batch/{}'.format(bucket, prefix, batch_file) 
sm_transformer.transform(input_location, split_type='Line')
sm_transformer.wait()

Using already existing model: linear-learner-2020-02-09-20-08-59-743


........................Docker entrypoint called with argument(s): serve
[02/09/2020 21:17:42 INFO 140403619051328] loaded entry point class algorithm.serve.server_config:config_api
[02/09/2020 21:17:42 INFO 140403619051328] loading entry points
[02/09/2020 21:17:42 INFO 140403619051328] loaded request iterator text/csv
[02/09/2020 21:17:42 INFO 140403619051328] loaded request iterator application/x-recordio-protobuf
[02/09/2020 21:17:42 INFO 140403619051328] loaded request iterator application/json
[02/09/2020 21:17:42 INFO 140403619051328] loaded request iterator application/jsonlines
[02/09/2020 21:17:42 INFO 140403619051328] loaded response encoder text/csv
[02/09/2020 21:17:42 INFO 140403619051328] loaded response encoder application/x-recordio-protobuf
[02/09/2020 21:17:42 INFO 140403619051328] loaded response encoder application/json
[02/09/2020 21:17:42 INFO 140403619051328] loaded response encoder application/jsonlines
[02/09/2020 21:17:42 INFO 140403619051328] loaded entry po

UnexpectedStatusException: Error for Transform job linear-learner-2020-02-09-21-13-49-246: Failed. Reason: ClientError: See job logs for more information

## Clean up

We will leave the prediction endpoint running at the end of this notebook so we can handle incoming event streams. However, don't forget to delete the prediction endpoint when you're done. You can do that at the Amazon SageMaker console in the Endpoints page. Or you can run `linear_predictor.delete_endpoint()`


## Data Acknowledgements

The dataset used to demonstrated the fraud detection solution has been collected and analysed during a research collaboration of Worldline and the Machine Learning Group (http://mlg.ulb.ac.be) of ULB (Université Libre de Bruxelles) on big data mining and fraud detection. More details on current and past projects on related topics are available on https://www.researchgate.net/project/Fraud-detection-5 and the page of the [DefeatFraud](https://mlg.ulb.ac.be/wordpress/portfolio_page/defeatfraud-assessment-and-validation-of-deep-feature-engineering-and-learning-solutions-for-fraud-detection/) project
We cite the following works:
* Andrea Dal Pozzolo, Olivier Caelen, Reid A. Johnson and Gianluca Bontempi. Calibrating Probability with Undersampling for Unbalanced Classification. In Symposium on Computational Intelligence and Data Mining (CIDM), IEEE, 2015
* Dal Pozzolo, Andrea; Caelen, Olivier; Le Borgne, Yann-Ael; Waterschoot, Serge; Bontempi, Gianluca. Learned lessons in credit card fraud detection from a practitioner perspective, Expert systems with applications,41,10,4915-4928,2014, Pergamon
* Dal Pozzolo, Andrea; Boracchi, Giacomo; Caelen, Olivier; Alippi, Cesare; Bontempi, Gianluca. Credit card fraud detection: a realistic modeling and a novel learning strategy, IEEE transactions on neural networks and learning systems,29,8,3784-3797,2018,IEEE
* Dal Pozzolo, Andrea Adaptive Machine learning for credit card fraud detection ULB MLG PhD thesis (supervised by G. Bontempi)
* Carcillo, Fabrizio; Dal Pozzolo, Andrea; Le Borgne, Yann-Aël; Caelen, Olivier; Mazzer, Yannis; Bontempi, Gianluca. Scarff: a scalable framework for streaming credit card fraud detection with Spark, Information fusion,41, 182-194,2018,Elsevier
* Carcillo, Fabrizio; Le Borgne, Yann-Aël; Caelen, Olivier; Bontempi, Gianluca. Streaming active learning strategies for real-life credit card fraud detection: assessment and visualization, International Journal of Data Science and Analytics, 5,4,285-300,2018,Springer International Publishing